In [34]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
from scipy import special

This script is getting the best choice of modulation indices by brute force alone.

This script determines the amplitudes for different tones depending on signals inputted into the EOM.
The aim is to work out these amplitudes for generic detuning factors. Then, with knowledge of the matrix elements for each transition, we will know what ratio we would like the amplitudes to take in order to have equal Rabi frequencies for each transition. Then, we can work backwards to find the modulation indices and thus the amplitudes of the electric fields for each modulating sine wave. These modulation indices should minimise the difference in Rabi frequency and maximise the efficiency.

This will initially only consider three modulating sine waves. This can be extended later on.

The maths an ideas for this code are written in OneNote.


Things that would be nice to change:
It would be nice to change from 3 modulation indices to N

In [35]:
#Define all of the frequencies we care about (we are working in MHz and are ignoring the 2pi for now to make things easy - also, frequencies are rounded) - these numbers don't actually matter.
#The setup is a bit difficult, so everything is carefully laid out:

DwG = 8000      #Ground state splitting
DwM = 60        #Metastable state splitting
Dw = 170.26e6   #Splitting between the lowest metastable state and the highest ground state

#Define where we have parked the laser relative to the 
Dw1 = 7000      #Where the unmodulated signal is sitting above the lowest ground state
Dw2 = 0       #Where the unmodulated signal is sitting above the lowest metastable state

#Define the frequency at which we have parked the beam
w = (DwG - Dw1) + Dw + Dw2

#Define the frequencies that we wish to find:
w1 = w - (DwG - Dw1) - Dw2
w2 = w - (DwG - Dw1) + (DwM - Dw2)
w3 = w + Dw1 - Dw2
w4 = w + Dw1 + (DwM - Dw2)

#Define the frequencies we wish to modulate at:
W = [(DwG - Dw1),Dw1,DwM]

#The aim is to have 4 equal Rabi frequencies and the most efficiency we can get. Set out the criteria for what we should aim for:
#Set the difference in Rabi frequencies to be 1 in 1000:
DiffRab = 1 / 1000
#Set the desired efficiency to be at least 10%:
DesiredEff = 0.1

In [36]:
#Work out how to make w1,w2,w3 and w4 using wm1,wm2 and wm3 from w. Consider only O orders.

Order = 3   #Considering up to the 2nd order.
values = range(-Order, Order + 1)  # from -Order to Order

Mod = []
#intertools produces all possible combinations of elements
for p in itertools.product(values, repeat = np.size(W)):     # this creates a list of all the possible combinations of each modulation frequency: we are making p a list rather than a tuple
    Mod.append(p)

#filter out all of the combinations that we won't use and create a list of all of the useful ones:
Combinations = []       #Define a list of all of the combinations for each frequency. Each component will be: [frequency that this adds up to, no. of W[0],...,no. of W[N]]
for list in Mod:

    #find the frequency when summing up the modulation frequencies as described by this component of Mod:
    freq = w
    for x in range(np.size(W)):
        freq += list[x] * W[x]

    #now, we want to see if this frequency is useful at all:
    if freq == w1:
        Combinations.append([1,list])
    if freq == w2:
        Combinations.append([2,list])
    if freq == w3:
        Combinations.append([3,list])
    if freq == w4:
        Combinations.append([4,list])

#so, now we should have a list of all of the useful combinations of the modulated frequencies:
print(Combinations)

[[1, (-1, 0, 0)], [2, (-1, 0, 1)], [3, (0, 1, 0)], [4, (0, 1, 1)]]


In [37]:
#Define a function that outputs the amplitude for this sideband:
def A(list,ModIndices):
    A = 1
    for x in range(np.size(ModIndices)):
        A *= special.jv(list[x], ModIndices[x])
    return(A)

#Define a function to find the amplitudes for each of the frequencies
def FindAmps(Combinations,ModIndices):
    Amps = [0,0,0,0]

    for i in range(4):
        for x in Combinations:
            if x[0] == i+1:
                Amps[i] += A(x[1],ModIndices)

    #Make the amplitudes absolute since their signs no longer matter (no more intereference):
    return(np.abs(Amps))


In [38]:
#Now, we have amplitudes but I assume that they are not what we want. We are starting from modulation indices defined at the top.
#We can use a bit of trial and error so that we can walk towards usable modulation indices.

#First of all, what is it that we want in the end? - set the matrix element for w1 to 1 and all of the others relative to this:
M = [1,1.1,0.9,1]

# I would like to define a function to minimise so that we can make an MCMC:

# this function finds the largest difference in the Rabi frequencies as a propotion of the mean of all of the Rabi frequencies
# also, for simplicity I have put the efficiency function in this as well so that we just have one function to run to get both
def DiffAndEff(Combinations,ModIndices,M):
    Amps = FindAmps(Combinations,ModIndices)

    mean = 0
    for x in range(np.size(M)):
        mean += Amps[x] * M[x]      #this adds up all of the Rabi frequencies
    mean /= np.size(M)              #then divide at the end to get the mean

    #now, find the largest difference:
    Diff = []

    for x in range(np.size(M)):
        Diff.append(np.abs(( Amps[0] * M[0] - Amps[x] * M[x] ) / mean ))  #to speed things up, we are just considering the difference between one of the Rabi frequencies and the others

    #find the maximum rabi freq out of all of these:
    MaxDiff = np.max(Diff)

    #also work out the efficiency:
    efficiency = 0
    for x in range(np.size(Amps)):
        efficiency += Amps[x] ** 2

    return(MaxDiff,efficiency)

In [ ]:
#if we start off with a larger mesh and see which of these points are best. Then we can zoom into the cube surrounding this point to see what works
Rinitial = [ [0,1.85] , [0,1.85] , [0,1.85] ]       #range to begin with
Spaces = [0.1,0.05,0.01,0.005]
# we can choose the lowest N points from each stage to then probe further. If any points satisfy our condition below then these should be saved.
MaxDiffAllowed = 0.001
N = 5

#define a function that returns points with the lowest difference in this range
def BestIndices(N,R,spaces,Combinations,MaxDiffAllowed):      #Here, we need R to define the minimum and maximum for x y and z

    #define all of the points we are calculating values at for every point in this mesh
    X = np.linspace(R[0][0], R[0][1], int( (R[0][1] - R[0][0]) / (spaces) + 1) )
    Y = np.linspace(R[1][0], R[1][1], int( (R[1][1] - R[1][0]) / (spaces) + 1) )
    Z = np.linspace(R[2][0], R[2][1], int( (R[2][1] - R[2][0]) / (spaces) + 1) )

    AllIndices = [] #save all of the indices
    AllDiffs = []  #save all of the differences

    Solutions = []

    for x in range(np.size(X)):
        for y in range(np.size(Y)):
            for z in range(np.size(Z)):
                ModIndices = [X[x],Y[y],Z[z]]       #the point we are considering 
                MaxDiff, _ = DiffAndEff(Combinations,ModIndices,M)
                AllIndices.append(ModIndices)
                AllDiffs.append(MaxDiff)

                #save any solutions
                if MaxDiff <= MaxDiffAllowed:
                    Solutions.append(ModIndices)

    # get indices of the three smallest diffs
    NextPoints = []
    for x in range(0,N):
        index = AllDiffs.index(min(AllDiffs))
        NextPoints.append( AllIndices[index] )
        del AllDiffs[index]
        del AllIndices[index]

    return(NextPoints)       #returning the three points that had the smallest difference in Rabi frequencies

In [40]:
#Define a function to call change indices into ranges
def IndicesToRanges(spacing,indices):
    #find the new ranges

    R = []

    # for each index we want to find the new points
    for index in indices:

        #find the new range by changing an index into a range
        RNew = []
        for i in range(0,np.size(np.array(index))):     #need this to be a np array to get the right size - probably is another way of doing this but idk
            RNew.append( [ index[i] - spacing , index[i] + spacing ] )
        R.append(RNew)

    return(R)


In [41]:
#find the best indices within a cube around these indices, and then a cube around those and so on:
#define a function that swaps old indices and old ranges into new indices and new ranges

def ZoomIn(OldIndices,OldRanges,N,NewSpacing,Combinations,MaxDiffAllowed):
    NewIndices = []
    for i in range(np.size(np.array(OldIndices)[:,0])):     #repeat for every index in the new indices
        temp = BestIndices(N,OldRanges[i],NewSpacing,Combinations,MaxDiffAllowed)

        #temp is a list of N lists of 3, so to append without appending in chunks of N lists, we would like to append like this:
        for j in range(0, int( np.size(temp) / np.size(temp[0]))): # this is the range of the number of sets of indices within temp
            NewIndices.append(temp[j])        #save these new indices in a larger list

    #convert these indices into ranges
    NewRanges = IndicesToRanges(NewSpacing,NewIndices)
        
    return(NewIndices,NewRanges)


In [42]:
#find the initial points

NewIndices1 = BestIndices(N,Rinitial,Spaces[0],Combinations,MaxDiffAllowed)
NewRanges1 = IndicesToRanges(Spaces[0],NewIndices1)

C:\Users\ollie\AppData\Local\Temp\ipykernel_28488\1271848334.py:23: RuntimeWarning: invalid value encountered in scalar divide
  Diff.append(np.abs(( Amps[0] * M[0] - Amps[x] * M[x] ) / mean ))  #to speed things up, we are just considering the difference between one of the Rabi frequencies and the others


In [43]:
#soom into these initial points and probe around them a bit, then zoom in again and again.
NewIndices2,NewRanges2 = ZoomIn(NewIndices1,NewRanges1,N,Spaces[1],Combinations,MaxDiffAllowed)
NewIndices3,NewRanges3 = ZoomIn(NewIndices2,NewRanges2,N,Spaces[2],Combinations,MaxDiffAllowed)
NewIndices4,NewRanges4 = ZoomIn(NewIndices3,NewRanges3,N,Spaces[3],Combinations,MaxDiffAllowed)

#this takes 44s when no solutions are found. (for N=3, spacing is 0.1,0.01,0.001)

C:\Users\ollie\AppData\Local\Temp\ipykernel_28488\1271848334.py:23: RuntimeWarning: invalid value encountered in scalar divide
  Diff.append(np.abs(( Amps[0] * M[0] - Amps[x] * M[x] ) / mean ))  #to speed things up, we are just considering the difference between one of the Rabi frequencies and the others


In [44]:
print(np.size(NewIndices3))

375


In [45]:
#we now want to find only points that satsfy our criteria and then find the one that has the highest efficiency.

Diffs = []
Effs = []
for i in range( 0, int( np.size( NewIndices3 ) / np.size(NewIndices4[0])) ):
    #find the efficiency and difference in rabi frequencies
    Diff, Eff = DiffAndEff(Combinations,NewIndices4[i],M)
    Diffs.append(Diff)
    Effs.append(Eff)

Diffs = np.array(Diffs, dtype=float)
Effs = np.array(Effs, dtype=float)

print(min(Diffs))

mask = Diffs <= 0.001
DiffResults = Diffs[mask]
EffResults = Effs[mask]

print(DiffResults)
print(EffResults)

1.151509911202814
[]
[]


In [46]:
print(M)
print(Spaces)

[1, 1.1, 0.9, 1]
[0.1, 0.05, 0.01, 0.005]
